In [87]:
from collections import defaultdict
import random

import numpy as np
import pandas as pd

import usaddress

In [3]:
# df_2020 = pd.read_excel('2015-2020 Evictions Maricopa County.xlsx', sheet_name='2020')
df_2020 = pd.read_csv('2015-2020 Evictions Maricopa County - 2020.csv')
df_2020.head(30)

/Users/yixiang/anaconda3/envs/pper/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,File Date,Case Number,SubCategory,Immediate,Court,Case Status,Case Status Date,Monetary Judgment,Judgment For,Address,...,Def2 Ph,Plaintiff Name,Plaintiff1 Address,Plaint1 Zip,Plaintiff1 Email,Plaintiff1 Ph,Plt FTA?,Plaintiff2 Name,Plaintiff2 Email,Plaintiff2 Ph
0,1/2/20,CC2020000490000,Poss Prop,N,Agua Fria,Adjudicated,1/10/20,"$3,653.95",Plaintiff,"10751 W. DAVIS LANE, AVONDALE",...,NaN,LSF9 MASTER PARTICIPATION TRUST,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
1,1/2/20,CC2020000495000,Poss Prop,N,Kyrene,Adjudicated,1/9/20,"$4,414.61",Plaintiff,"5102 E PIEDMONT ROAD 2244, PHOENIX",...,NaN,FPA/WC PINNACLE PHASE I LLC,"5151 E GUADALUPE ROAD, PHOENIX AZ",85044,NaN,(480) 496-6161,N,NaN,NaN,NaN
2,1/2/20,CC2020000496000,Poss Prop,N,San Marcos,Adjudicated,1/9/20,"$6,098.23",Plaintiff,"3800 W CHANDLER BLVD 1297, CHANDLER",...,NaN,GS CHANDLER LLC,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
3,1/2/20,CC2020000497000,Residential,N,San Marcos,Adjudicated,1/9/20,"$2,823.59",Plaintiff,"2800 N ARIZONA AVE 234, CHANDLER",...,NaN,DESERT JEWEL APARTMENTS LLC,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
4,1/2/20,CC2020000498000,Rent,N,University Lakes,Adjudicated,1/9/20,"$5,578.58",Plaintiff,"1720 E Broadway rd 2127, Tempe",...,NaN,Marbeya,"4222 E Thomas Rd #230, Phoenix AZ",85018,NaN,NaN,N,NaN,NaN,NaN
5,1/2/20,CC2020000500000,Poss Prop,N,San Marcos,Adjudicated,1/9/20,"$5,712.99",Plaintiff,"3950 W CHANDLER BLVD 2072, CHANDLER",...,NaN,RPC CHAMDLER APARTMENTS LLC,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
6,1/2/20,CC2020000501000,Poss Prop,N,Maryvale,Adjudicated,1/10/20,"$1,604.50",Plaintiff,"6301 N 64th Dr 11, Phoenix",...,NaN,ACKK Enterprises LLC,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
7,1/2/20,CC2020000502000,Rent,N,Country Meadows,Adjudicated,1/8/20,"$3,274.59",Plaintiff,"12175 W MCDOWELL RD 1308, AVONDALE",...,NaN,ASHTON POINTE HOLDINGS LLC,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
8,1/2/20,CC2020000503000,Poss Prop,N,Maryvale,Adjudicated,1/6/20,NaN,NaN,"4030 N 44th Ave 2077, Phoenix",...,NaN,Cove Phoenix LLC,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
9,1/2/20,CC2020000508000,Rent,N,Arrowhead,Adjudicated,1/9/20,"$3,961.02",Plaintiff,"8092 W PARADISE LN 1140, PEORIA",...,NaN,ENCLAVE INVESTORS LP,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN


In [243]:
df_2020[df_2020['Case Number']=='CC2020000495000']['Address'], df_2020[df_2020['Case Number']=='CC2020000495000']['Defendant1 Address']

(1    5102 E PIEDMONT ROAD 2244, PHOENIX
 Name: Address, dtype: object,
 1    5102 E PIEDMONT ROAD, PHOENIX  AZ
 Name: Defendant1 Address, dtype: object)

In [4]:
df_2020.shape[0]

37760

In [5]:
df_2020.columns

Index(['File Date', 'Case Number', 'SubCategory', 'Immediate', 'Court',
       'Case Status', 'Case Status Date', 'Monetary Judgment', 'Judgment For',
       'Address', 'Zip', 'Writ Date', 'Plt Legal Rep?', 'Def Legal Rep?',
       'Claim Amount', 'Bench Trial?', 'Defendant1 Name', 'Defendant1 Address',
       'Def1 Zip', 'Def1 Email', 'Deft1 Ph', 'Def FTA?', 'Defendant2 Name',
       'Def2 Email', 'Def2 Ph', 'Plaintiff Name', 'Plaintiff1 Address',
       'Plaint1 Zip', 'Plaintiff1 Email', 'Plaintiff1 Ph', 'Plt FTA?',
       'Plaintiff2 Name', 'Plaintiff2 Email', 'Plaintiff2 Ph'],
      dtype='object')

In [6]:
df_2020.groupby(['Court'])['Court'].count()

Court
Agua Fria             638
Arcadia Biltmore     2355
Arrowhead             559
Country Meadows      3056
Desert Ridge         1166
Downtown              963
Dreamy Draw          1787
East Mesa            1316
Encanto              2052
Hassayampa            720
Highland              748
Ironwood               16
Kyrene               2096
Manistee             2924
Maryvale             2214
McDowell Mountain    1036
Moon Valley          2482
North Mesa           1356
North Valley         1141
San Marcos           1673
San Tan               655
South Mountain       1348
University Lakes     1412
West McDowell        1526
West Mesa            1701
White Tank            820
Name: Court, dtype: int64

## Defendants

Related attributes: 'Defendant1 Name', 'Defendant1 Address', 'Def1 Zip', 'Def1 Email', 'Deft1 Ph', 'Def FTA?', 'Defendant2 Name', 'Def2 Email', 'Def2 Ph'

In [7]:
defendants1 = df_2020[['Defendant1 Name']].dropna()['Defendant1 Name'].tolist()
defendants2 = df_2020[['Defendant2 Name']].dropna()['Defendant2 Name'].tolist()
len(defendants1), len(defendants2), len(defendants1 + defendants2)

(37753, 12185, 49938)

In [122]:
dedup_defendants1 = set(map(lambda x: x.lower(), defendants1))
dedup_defendants2 = set(map(lambda x: x.lower(), defendants2))
dedup_all_defendants = dedup_defendants1 | dedup_defendants2
len(dedup_defendants1), len(dedup_defendants2), len(dedup_all_defendants)

(30230, 10688, 38590)

In [240]:
len(defendants1) - len(dedup_defendants1), len(defendants2) - len(dedup_defendants2), len(defendants1 + defendants2) - len(dedup_all_defendants)

(7523, 1497, 11348)

In [131]:
# import nltk
# from nltk.tag.stanford import NERTagger
# st = NERTagger('stanford-ner/all.3class.distsim.crf.ser.gz', 'stanford-ner/stanford-ner.jar')

# def is_person_name(text):
#     for sent in nltk.sent_tokenize(text):
#         tokens = nltk.tokenize.word_tokenize(sent)
#         tags = st.tag(tokens)
#         for tag in tags:
#             if tag[1]=='PERSON': print tag

In [137]:
# names with punctuations
from string import punctuation
for d in dedup_all_defendants:
    if any(p in d for p in punctuation):
        print(d)

marquetta garrett-mitchell
zi-yah thompson
traci williams-medley
catherine o'connor
jo-anne cassity
caitlyn metcalf-boat
melanie nez-armstrong
samantha rojas-guevara
irma nunez-espinoza
khamari grady-cloud
karl adam-narvesen
jacklynn d'agay
sabrina congdon-lervold
james o'kelley
rekee-terasheena walker mcgary
tenae griffin-reason
de'on roberts
r.tudor investments llc
sumer al-fayez
m. whitehead
marion vasquez-sanchez
nevyn ivers-frank
de'shon allen
w. billy king
bunetta gillespie-randles
michael stout-squires
daniel valencia-cervantes
racquel winkler-smith
taylor hall-brown
kelly byers-gustafson
petrocelli ciccione-jimenez
alexandra wester-blanco
yolanda ray-alexander
jeanne c'de baca
jose ochoa- carrera
anna-ellen graham-begay
ting-hsuan kuo
rosalind ellis- mcghee
ta'lor corely
laurie-lynn soares
takia-denise stewart
katelynn krpata-caron
yerania chavez-ruiz
melissa oviedo-owens
dr. robbins bibbs
krista scheinost-greening
j.r. benton
monique battee-freeman
danny marty-rodriguez
shelly

In [139]:
# names longer than 5 tokens
# companies: llc, co.
for d in dedup_all_defendants:
    if len(d.split(' ')) > 5:
        print(d)

the estate of george speck, the estate of ethel speck
antonio de jesus corona et al
the estate of leslie r weinberger
venture well drilling, pump co. llc
the estate of jim bohanon, sonia m carano
the estate of joyce m. doran
bp saratoga ridge ut 4 llc
all tenents and occupants and homeless
the estate of john michael beausoleil
med shape weight loss clinic, llc
miram hutchinson irrevocable living trust march
jason coleman / sweetwater behavioral care


In [155]:
for d in dedup_all_defendants:
    if '.' in d:  # . -
        print(d)

r.tudor investments llc
m. whitehead
w. billy king
dr. robbins bibbs
j.r. benton
ms. jordan arbo
wanderjaunt inc.
dr. ludmila pyter
joh.shay tinsley
dennard higgins sr.
v. lachaal
v. audreana bloodsaw
venture well drilling, pump co. llc
lene st. john
bader a. alkhaldi
r. trep thompson
samantha h.
katie st. louis
joshua st. clair
.99 super center ymas
estate of patricia a. fautenberry
the estate of joyce m. doran
skilah arnol.d
wanderjaunt, inc.
christopher st. armour
charles simpson jr.


In [222]:
dedup_all_defendants_list = list(dedup_all_defendants)

# tokenization and blocking
def tokenize(name):
    return list(set(name.split(' ')) - set(punctuation))

defendant_ii = defaultdict(set)
for idx, name in enumerate(dedup_all_defendants_list):
    tokens = tokenize(name)
    for t in tokens:
        defendant_ii[t].add(idx)

In [223]:
len(defendant_ii.keys())

22761

In [224]:
# # of full comparison, 3 of blocked comps
full_comps = len(dedup_all_defendants) * (len(dedup_all_defendants)-1)
blocked_comps = sum([len(v)*(len(v)-1) for k, v in defendant_ii.items()])
full_comps, blocked_comps

(1489149510, 3721514)

In [225]:
# need a more effective blocking method? RR is already:
f'RR: {round(100 * (1 - blocked_comps / full_comps), 2)}%'

'RR: 99.75%'

In [239]:
import rltk
for idx, (k, v) in enumerate(defendant_ii.items()):
    v = list(v)
    for i in range(len(v)):
        for j in range(i+1, len(v)):
            r1 = dedup_all_defendants_list[v[i]]
            r2 = dedup_all_defendants_list[v[j]]
            sim = rltk.monge_elkan_similarity(tokenize(r1), tokenize(r2))
            if sim > 0.9:
                print(r1, '|', r2, '|', sim)
    if idx > 5:
        break

xavier piedra | xavier pierce | 0.9222222222222222
xavier pierce | xavier perez | 0.92
kristin bell | tristen bell | 0.9047619047619049
kristin bell | christina bell | 0.9206349206349207
branden bell | breanna bell | 0.9396825396825397
lorene bell | bell leon | 0.9125
lorene bell | lawrence bell | 0.9187500000000001
christina bell | christopher bell | 0.9212121212121211
ajia bell | ariya bell | 0.9025
vanessa bell | jhanessa bell | 0.9345238095238095
steve bell | steven bell | 0.9833333333333334
jhanessa bell | jessica bell | 0.9008928571428572
aaron johnson | aaron jones | 0.9161904761904762
aaron marin | aaron martin | 0.9805555555555556
aaron burton | aaron brown | 0.92
aaron stone | aaron sutton | 0.92
aaron stone | aaron stowers | 0.9266666666666666
aaron morand | aaron jordan | 0.9111111111111112
aaron williamson | aaron yates williams | 0.98
aaron anderson | aaron zanders | 0.9345238095238095
aaron mcmillan | aaron mcmillian | 0.9888888888888889
aaron browning | aaron brown | 0.

### Compare address

In [10]:
df_2020[df_2020['Address'] == df_2020['Defendant1 Address']]

,File Date,Case Number,SubCategory,Immediate,Court,Case Status,Case Status Date,Monetary Judgment,Judgment For,Address,...,Def2 Ph,Plaintiff Name,Plaintiff1 Address,Plaint1 Zip,Plaintiff1 Email,Plaintiff1 Ph,Plt FTA?,Plaintiff2 Name,Plaintiff2 Email,Plaintiff2 Ph


In [11]:
df_address = df_2020.apply(lambda x: x['Address'][:10] if not pd.isna(x['Address']) else None, axis=1)
df_defendant1_address = df_2020.apply(lambda x: x['Defendant1 Address'][:10] if not pd.isna(x['Defendant1 Address']) else None, axis=1)

In [12]:
# how many cases whose first ten letters of df_address 
# are the same as the first ten letters of df_defendant1_address.
df_2020[df_address == df_defendant1_address]['Case Number'].shape[0]

34218

In [13]:
# how many of them are different
df_2020['Case Number'].shape[0] - df_2020[df_address == df_defendant1_address]['Case Number'].shape[0]

3542

In [14]:
# df_compare = pd.concat(
#     {'Address': df_address, 'Defendant1 Address': df_defendant1_address}, axis=1)
# df_compare = df_compare.reset_index(drop=True)
# df_compare

In [17]:
all_addresses = set(filter(lambda x: not pd.isna(x), df_2020['Address'].to_list()))
all_addresses

{'829 E TURNEY AVE UNIT 1, PHX',
 '4141 N 55TH AVE  1250, PHOENIX',
 '420 N Scottsdale Rd 6024, Tempe',
 '1450 S COOPER RD 2059, CHANDLER',
 '13421 N 43rd Ave #3067, Phoenix',
 '1460 E BELL ROAD 2086-1, PHOENIX',
 '4221 W DUNLAP AVE 235, PHX',
 '4630 S Lakeshore Drive 125, Tempe',
 '13300 N 88TH AVE UNIT 3014, PEORIA',
 '5150 W EUGIE AVENUE 2011, GLENDALE',
 '17600 N. 79TH AVE. (GATE CODE #1606) UNIT #508, GLENDALE',
 '1119 N 9TH STREET, PHOENIX',
 '538 N 55TH AVE 1031, GLENDALE',
 '6755 N 83RD AVE 15205, GLENDALE',
 '11459 NORTH 28TH DRIVE 3002, PHOENIX',
 '6505 E OSBORN RD UNIT 215-1, SCOTTSDALE',
 '34000 N 27th Dr #3116, Phoenix',
 '2145 W BROADWAY ROAD 251, MESA',
 '6901 W MCDOWELL RD 8204, PHOENIX',
 '7340 E LEGACY BLVD UNIT J1004, SCOTTSDALE',
 '24840 W DOVE TRAIL, BUCKEYE',
 '1033 S. LONGMORE ST #1094, MESA',
 '8520 N 54TH DRIVE, GLENDALE',
 '4002 E SOUTHERN AVE UNIT 249, PHOENIX',
 '1829 N 45th Ave Unit# 2128-2, Phoenix',
 '8600 E BROADWAY RD  #0109, MESA',
 '7007 W INDIAN SCHO

In [173]:
usaddress.tag('7375 N ZANJERO BLVD 5-1009, GLENDALE')[0]

OrderedDict([('AddressNumber', '7375'),
             ('StreetNamePreDirectional', 'N'),
             ('StreetName', 'ZANJERO'),
             ('StreetNamePostType', 'BLVD'),
             ('OccupancyIdentifier', '5-1009'),
             ('PlaceName', 'GLENDALE')])

In [216]:
usaddress.parse('1401 NORTH 43RD AVENUE  UNIT # 2012, PHOENIX')

[('1401', 'AddressNumber'),
 ('NORTH', 'StreetNamePreDirectional'),
 ('43RD', 'StreetName'),
 ('AVENUE', 'StreetNamePostType'),
 ('UNIT', 'OccupancyType'),
 ('#', 'OccupancyIdentifier'),
 ('2012,', 'OccupancyIdentifier'),
 ('PHOENIX', 'PlaceName')]

In [212]:
address_tag_content = defaultdict(set)
address_parsed = []
# for addr in all_addresses:
#     try:
#         parsed_addr = usaddress.tag(addr)[0]
#     except usaddress.RepeatedLabelError as e:
#         print('-' * 10)
#         print('RepeatedLabelError:', e)
#         print('Original:', e.original_string)
#         print('Parsed:', e.parsed_string)
#     for content, tag in parsed_addr.items():
#         address_tag_content[tag].add(content)

for addr in all_addresses:
    parsed_addr = usaddress.parse(addr)
    for content, tag in parsed_addr:
        address_tag_content[tag].add(content)
    address_parsed.append({'original': addr, **{'a':1 for pa in parsed_addr}})

In [213]:
address_tag_content.keys()

dict_keys(['AddressNumber', 'StreetNamePreDirectional', 'StreetName', 'StreetNamePostType', 'OccupancyType', 'OccupancyIdentifier', 'PlaceName', 'SubaddressType', 'SubaddressIdentifier', 'ZipCode', 'CountryName', 'StreetNamePreType', 'StreetNamePostDirectional', 'StateName', 'AddressNumberPrefix', 'USPSBoxType', 'USPSBoxID', 'AddressNumberSuffix', 'USPSBoxGroupType', 'BuildingName', 'Recipient', 'IntersectionSeparator'])

In [214]:
# address_tag_content['StreetNamePreDirectional']

In [215]:
# address_tag_content['StreetNamePostType']

In [81]:
df_address = pd.DataFrame(address_parsed)
df_address

,original,AddressNumber,StreetNamePreDirectional,StreetName,StreetNamePostType,OccupancyType,OccupancyIdentifier,PlaceName,SubaddressType,SubaddressIdentifier,...,StreetNamePostDirectional,StateName,AddressNumberPrefix,USPSBoxType,USPSBoxID,AddressNumberSuffix,USPSBoxGroupType,BuildingName,Recipient,IntersectionSeparator
0,"829 E TURNEY AVE UNIT 1, PHX",829,E,TURNEY,AVE,UNIT,PHX,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"4141 N 55TH AVE 1250, PHOENIX",4141,N,55TH,AVE,NaN,"1250,",PHOENIX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"420 N Scottsdale Rd 6024, Tempe",420,N,Scottsdale,Rd,NaN,"6024,",Tempe,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"1450 S COOPER RD 2059, CHANDLER",1450,S,COOPER,RD,NaN,"2059,",CHANDLER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"13421 N 43rd Ave #3067, Phoenix",13421,N,43rd,Ave,NaN,"3067,",Phoenix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32342,"12930 W PARADISE DR, EL MIRAGE",12930,W,PARADISE,"DR,",NaN,NaN,MIRAGE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32343,"1330 W BROADWAY RD A-104, TEMPE",1330,W,BROADWAY,RD,TEMPE,"A-104,",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32344,"5125 N. 40TH ST. F216, PHOENIX",5125,N.,40TH,ST.,NaN,"F216,",PHOENIX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32345,"1950 N 43rd Ave Unit# 1047-1, Phoenix",1950,N,43rd,Ave,Unit,"1047-1,",Phoenix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
random.seed(2021)
df_address_samples = pd.DataFrame.from_dict(
    {tag: [random.sample(content, k=min(5, len(content)))] for tag, content in address_tag_content.items()},
    orient='index',
    columns=['Samples']
)

In [115]:
df_address_stat = pd.DataFrame(df_address.count(), columns=['Count'])
df_address_stat['%'] = df_address_stat.apply(lambda x: round(100.0 * x / df_address.shape[0], 2))
df_address_stat = df_address_stat.join(df_address_samples)
df_address_stat.sort_values(by=['Count'], ascending=False)

,Count,%,Samples
original,32347,100.00,NaN
StreetName,32334,99.96,"[SOL,, LONGVIEW, 209th, SPANISH, 92ST]"
AddressNumber,32332,99.95,"[10769, 1325, 22629, 15431W, 31155]"
StreetNamePreDirectional,32099,99.23,"[NE, W,, w., SOUTH, M.]"
PlaceName,30608,94.62,"[wickenberg, PHOOENIX, PHOENNIX, MIRAGE, mesa]"
StreetNamePostType,30549,94.44,"[road,, AVENUE,, RE, BLVD.,, pl,]"
OccupancyIdentifier,26200,81.00,"[189A,, 1011,, 2144,, E105,, 7724-07,]"
OccupancyType,9036,27.93,"[tempe, OHX, UNITN, Apt., Drve]"
StreetNamePreType,821,2.54,"[BRILL, McDowell, ROAD, CAMINO, KATHLEEN]"
StreetNamePostDirectional,153,0.47,"[E,, WEST, SE,, ve, EAST]"


In [120]:
# need data cleaning
for addr in address_parsed:
    if 'AddressNumber' in addr and addr['AddressNumber'] == '15431W':
        print(addr)
        break

{'original': '15431W DESERT MIRAGE DR, SURPRISE', 'AddressNumber': '15431W', 'StreetName': 'MIRAGE', 'StreetNamePostType': 'DR,', 'PlaceName': 'SURPRISE'}


## Plaintiffs

Related attributes: 'Plaintiff Name', 'Plaintiff1 Address', 'Plaint1 Zip', 'Plaintiff1 Email', 'Plaintiff1 Ph', 'Plt FTA?', 'Plaintiff2 Name', 'Plaintiff2 Email', 'Plaintiff2 Ph'

In [167]:
plaintiff1 = df_2020[['Plaintiff Name']].dropna()['Plaintiff Name'].tolist()
plaintiff2 = df_2020[['Plaintiff2 Name']].dropna()['Plaintiff2 Name'].tolist()
len(plaintiff1), len(plaintiff2), len(plaintiff1 + plaintiff2)

(37760, 1700, 39460)